# Extracting the data b/w neural activity and images

In [1]:
! pip install --upgrade caveclient

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.7/66.7 kB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for caveclient: filename=caveclient-5.15.2-py3-none-any.whl size=75810 sha256=862cfd3940fcab691f2bc8679e6e64e4c022cc544f6b1218bf17c055a2f3b2a5
  Stored in directory: /home/vscode/.cache/pip/wheels/69/fc/fc/75346c997650659945c7828e277ef5015575b23f5d5a7133be
Successfully built caveclient
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
microns-phase3 0.8.0 requires caveclient==5.7.0, but you have caveclient 5.15.2 which is incompatible.

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [1]:
import numpy as np
import pandas as pd
import datajoint as dj
from caveclient import CAVEclient
from microns_phase3 import nda, utils
import matplotlib.pyplot as plt

Connecting microns@db.datajoint.com:3306


Could not create the ~log table
Could not access table `microns_phase3_nda`.`~log`
Could not create the ~log table
Could not access table `microns_phase3_nda`.`~log`


In [2]:
client = CAVEclient()
client.auth.save_token("64ea0271652fd86dc7fb6376cd9d52b9")

In [3]:
client = CAVEclient('minnie65_public')
client.materialize.version = 661

In [5]:
client.materialize.get_tables()

['synapses_pni_2',
 'baylor_gnn_cell_type_fine_model_v2',
 'nucleus_alternative_points',
 'connectivity_groups_v507',
 'proofreading_status_public_release',
 'allen_column_mtypes_v1',
 'allen_v1_column_types_slanted_ref',
 'aibs_column_nonneuronal_ref',
 'nucleus_ref_neuron_svm',
 'aibs_soma_nuc_exc_mtype_preds_v117',
 'baylor_log_reg_cell_type_coarse_v1',
 'apl_functional_coreg_forward_v5',
 'nucleus_detection_v0',
 'aibs_soma_nuc_metamodel_preds_v117',
 'coregistration_manual_v3']

In [13]:
in_ex_df = client.materialize.query_table('baylor_log_reg_cell_type_coarse_v1')

In [15]:
matched_df = client.materialize.query_table('coregistration_manual_v3')

In [14]:
in_ex_df.head()

,id_ref,created_ref,valid_ref,volume,pt_supervoxel_id,pt_root_id,id,created,valid,target_id,classification_system,cell_type,pt_position,bb_start_position,bb_end_position
0,17115,2020-09-28 22:41:18.237823+00:00,t,268.646482,75934403318291307,864691135635239593,25718,2023-03-22 18:05:52.744496+00:00,t,17115,baylor_log_reg_cell_type_coarse,inhibitory,"[80992, 109360, 15101]","[nan, nan, nan]","[nan, nan, nan]"
1,17816,2020-09-28 22:42:54.932823+00:00,t,264.795587,75090047309035210,864691135618175635,25581,2023-03-22 18:05:52.650844+00:00,t,17816,baylor_log_reg_cell_type_coarse,inhibitory,"[74880, 110032, 16883]","[nan, nan, nan]","[nan, nan, nan]"
2,18023,2020-09-28 22:43:00.306675+00:00,t,264.791327,75934266147628505,864691135207734905,5033,2023-03-22 18:04:23.575096+00:00,t,18023,baylor_log_reg_cell_type_coarse,inhibitory,"[81008, 108240, 16995]","[nan, nan, nan]","[nan, nan, nan]"
3,18312,2020-09-28 22:44:09.407821+00:00,t,221.584753,75441272688753483,864691135758479438,32294,2023-03-22 18:06:11.872068+00:00,t,18312,baylor_log_reg_cell_type_coarse,inhibitory,"[77392, 105280, 17650]","[nan, nan, nan]","[nan, nan, nan]"
4,255686,2020-09-28 22:40:42.632533+00:00,t,297.846047,88954888800920543,864691135568539372,2693,2023-03-22 18:04:21.985021+00:00,t,255686,baylor_log_reg_cell_type_coarse,excitatory,"[175760, 126480, 15504]","[nan, nan, nan]","[nan, nan, nan]"


In [16]:
matched_df.head()

,id,created,valid,target_id,session,scan_idx,unit_id,field,residual,score,id_ref,created_ref,valid_ref,volume,pt_supervoxel_id,pt_root_id,pt_position,bb_start_position,bb_end_position
0,13434,2023-04-05 22:39:53.474202+00:00,t,582276,7,5,4909,4,6.10722,9.568690,582276,2020-09-28 22:41:42.536121+00:00,t,274.291589,111332012547320963,864691135738685297,"[338720, 125232, 19589]","[nan, nan, nan]","[nan, nan, nan]"
1,3045,2023-04-05 22:39:03.110911+00:00,t,194144,7,4,9575,6,8.55708,-0.714900,194144,2020-09-28 22:42:01.511773+00:00,t,213.307228,83542405709639148,864691135614842827,"[136400, 170640, 17951]","[nan, nan, nan]","[nan, nan, nan]"
2,1474,2023-04-05 22:36:19.477499+00:00,t,293318,9,4,8066,6,5.63758,8.915025,293318,2020-09-28 22:45:23.572432+00:00,t,430.108180,90783515419343652,864691135502985397,"[189472, 119296, 26037]","[nan, nan, nan]","[nan, nan, nan]"
3,7915,2023-04-05 22:39:15.451639+00:00,t,332833,6,7,6413,6,5.27806,7.808064,332833,2020-09-28 22:44:41.864456+00:00,t,274.418729,93535317407139914,864691135387371905,"[209328, 174304, 20004]","[nan, nan, nan]","[nan, nan, nan]"
4,11589,2023-04-05 22:39:47.420305+00:00,t,295460,9,3,7748,6,5.20837,2.999543,295460,2020-09-28 22:41:24.368712+00:00,t,208.616653,90225718084406516,864691135661410544,"[185088, 157776, 15691]","[nan, nan, nan]","[nan, nan, nan]"


In [19]:
in_ex_id = np.unique(in_ex_df['pt_root_id'])
matched_id = np.unique(matched_df['pt_root_id'])

## Continuare dopo

In [11]:
unit_keys = matched_df[['session', 'scan_idx', 'unit_id']].to_dict(orient='records')

13925

In [14]:
nframes, fps = (nda.Scan & unit_keys).fetch1('nframes', 'fps')  # fetch # frames and fps
time_axis = np.arange(nframes)/ fps # create time axis (seconds)
spike_trace = (nda.Activity & unit_keys).fetch1('trace') # fetch spike trace
calcium_trace = (nda.ScanUnit * nda.Fluorescence & unit_keys).fetch1('trace') # fetch calcium fluorescence trace
pupil_radius = (nda.ManualPupil & unit_keys).fetch1('pupil_maj_r') # fetch manually segmented pupil trace 

KeyboardInterrupt: 

In [17]:
nframes, fps = (nda.Scan & unit_keys).fetch('nframes', 'fps')  # fetch # frames and fps

In [19]:
nframes_list = []
fps_list = []
for k in unit_keys:
    nframes, fps = (nda.Scan & k).fetch1('nframes', 'fps')
    nframes_list.append(nframes)
    fps_list.append(nframes)

KeyboardInterrupt: 

In [ ]:
len(nframes_list)v